# 🧠 Neural Prompt Parser Training (DistilBERT)

**Guitar Strum Generator - Master Thesis**  
**Author:** Rohan Rajendra Dhanawade

This notebook trains a **DistilBERT-based neural network** to extract musical features from natural language prompts.

## What is DistilBERT?
- A **66 million parameter** transformer model
- Pre-trained on **billions of words** (Wikipedia + BookCorpus)
- Understands **semantic meaning** of text
- We **fine-tune** it on YOUR music dataset

## Architecture
```
Input: "moody atmospheric indie track"
         │
         ▼
┌─────────────────────────┐
│    DistilBERT Encoder   │  ← 66M parameters, pre-trained
│   (6 transformer layers)│
└───────────┬─────────────┘
            │
    [CLS] embedding (768-dim)
            │
      ┌─────┴─────┐
      │           │
      ▼           ▼
┌──────────┐ ┌──────────┐
│ Emotion  │ │  Genre   │
│  Head    │ │  Head    │
└────┬─────┘ └────┬─────┘
     │            │
     ▼            ▼
"melancholic"   "indie"
```

## Step 1: Setup

In [ ]:
# Install dependencies (Colab has most pre-installed)
!pip install transformers -q

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
import json
import numpy as np
from typing import Dict, List, Tuple
from google.colab import files

# Check GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")
if device == 'cuda':
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Step 2: Upload Your Dataset

Upload your `train.jsonl` file from the Chat 4 dataset construction.

In [ ]:
# Option 1: Upload from your computer
print("Upload your train.jsonl file:")
uploaded = files.upload()

# Show uploaded file
for filename in uploaded.keys():
    print(f"Uploaded: {filename}")

In [ ]:
# Option 2: Clone from your GitHub (alternative)
# !git clone https://github.com/rohand575/guitar-strum-gen.git
# TRAIN_PATH = 'guitar-strum-gen/data/processed/train.jsonl'

# If you uploaded, use this:
TRAIN_PATH = 'train.jsonl'

## Step 3: Define Constants and Label Mappings

In [ ]:
# These MUST match your schema.py!
VALID_EMOTIONS = [
    'upbeat', 'melancholic', 'mellow', 'energetic',
    'peaceful', 'dramatic', 'hopeful', 'nostalgic'
]

VALID_GENRES = [
    'pop', 'rock', 'folk', 'ballad', 'country',
    'blues', 'jazz', 'indie', 'acoustic'
]

# Create label mappings
EMOTION_TO_IDX = {e: i for i, e in enumerate(VALID_EMOTIONS)}
IDX_TO_EMOTION = {i: e for e, i in EMOTION_TO_IDX.items()}

GENRE_TO_IDX = {g: i for i, g in enumerate(VALID_GENRES)}
IDX_TO_GENRE = {i: g for g, i in GENRE_TO_IDX.items()}

NUM_EMOTIONS = len(VALID_EMOTIONS)  # 8
NUM_GENRES = len(VALID_GENRES)      # 9

print(f"Emotion classes: {NUM_EMOTIONS}")
print(f"Genre classes: {NUM_GENRES}")

## Step 4: Create PyTorch Dataset

In [ ]:
class PromptDataset(Dataset):
    """
    Dataset for training the neural prompt parser.
    
    Converts JSONL samples into tensors:
    - input_ids: Tokenized prompt
    - attention_mask: Which tokens are real
    - emotion_label: Integer class (0-7)
    - genre_label: Integer class (0-8)
    """
    
    def __init__(self, data_path: str, tokenizer, max_length: int = 128):
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.samples = []
        
        with open(data_path, 'r') as f:
            for line in f:
                self.samples.append(json.loads(line))
        
        print(f"Loaded {len(self.samples)} samples")
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        sample = self.samples[idx]
        
        # Tokenize prompt
        encoding = self.tokenizer(
            sample['prompt'],
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )
        
        # Get labels
        emotion_label = EMOTION_TO_IDX.get(
            sample.get('emotion', 'mellow').lower(),
            EMOTION_TO_IDX['mellow']
        )
        genre_label = GENRE_TO_IDX.get(
            sample.get('genre', 'pop').lower(),
            GENRE_TO_IDX['pop']
        )
        
        return {
            'input_ids': encoding['input_ids'].squeeze(0),
            'attention_mask': encoding['attention_mask'].squeeze(0),
            'emotion_label': torch.tensor(emotion_label, dtype=torch.long),
            'genre_label': torch.tensor(genre_label, dtype=torch.long)
        }

## Step 5: Define the Neural Model (DistilBERT + Classification Heads)

In [ ]:
class DistilBertPromptParser(nn.Module):
    """
    Neural Prompt Parser using DistilBERT.
    
    This is a REAL neural network:
    - 66M parameters in DistilBERT encoder
    - Classification heads for emotion and genre
    - Trained on YOUR music dataset!
    """
    
    def __init__(self, dropout_rate: float = 0.3, freeze_bert: bool = False):
        super().__init__()
        
        # Load pre-trained DistilBERT
        print("Loading DistilBERT (66M parameters)...")
        self.bert = DistilBertModel.from_pretrained('distilbert-base-uncased')
        
        if freeze_bert:
            for param in self.bert.parameters():
                param.requires_grad = False
            print("BERT frozen (feature extraction mode)")
        else:
            print("BERT trainable (fine-tuning mode)")
        
        hidden_size = self.bert.config.hidden_size  # 768
        
        # Emotion classification head
        self.emotion_classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, NUM_EMOTIONS)
        )
        
        # Genre classification head  
        self.genre_classifier = nn.Sequential(
            nn.Linear(hidden_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, NUM_GENRES)
        )
        
        # Print parameter counts
        bert_params = sum(p.numel() for p in self.bert.parameters())
        head_params = sum(p.numel() for p in self.emotion_classifier.parameters())
        head_params += sum(p.numel() for p in self.genre_classifier.parameters())
        print(f"DistilBERT params: {bert_params:,}")
        print(f"Classification heads: {head_params:,}")
        print(f"Total: {bert_params + head_params:,}")
    
    def forward(self, input_ids, attention_mask):
        # Get BERT embeddings
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        
        # Use [CLS] token (index 0) as sentence representation
        cls_embedding = outputs.last_hidden_state[:, 0, :]  # [batch, 768]
        
        # Classification
        emotion_logits = self.emotion_classifier(cls_embedding)
        genre_logits = self.genre_classifier(cls_embedding)
        
        return emotion_logits, genre_logits

## Step 6: Initialize Model and Tokenizer

In [ ]:
# Load tokenizer
print("Loading tokenizer...")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Demo tokenization
demo_text = "moody atmospheric indie track"
tokens = tokenizer.tokenize(demo_text)
print(f"\nDemo tokenization:")
print(f"  Input: '{demo_text}'")
print(f"  Tokens: {tokens}")
print(f"  Token IDs: {tokenizer.encode(demo_text)}")

In [ ]:
# Initialize model
model = DistilBertPromptParser(freeze_bert=False)
model.to(device)

print(f"\nModel on: {device}")

## Step 7: Create Dataset and DataLoader

In [ ]:
# Create dataset
train_dataset = PromptDataset(TRAIN_PATH, tokenizer)

# Create dataloader
BATCH_SIZE = 16
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

print(f"\nBatches per epoch: {len(train_loader)}")

## Step 8: Training Loop 🔥

**This is where the LEARNING happens!**

The model:
1. Reads prompts from your dataset
2. Makes predictions (emotion, genre)
3. Compares to ground truth labels
4. Calculates loss (how wrong it was)
5. **Backpropagates** to adjust 66M+ weights
6. Repeats until accurate

In [ ]:
# Training hyperparameters
EPOCHS = 10
LEARNING_RATE = 2e-5  # Standard for BERT fine-tuning

# Loss function (Cross-Entropy for classification)
criterion = nn.CrossEntropyLoss()

# Optimizer (AdamW is standard for transformers)
optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)

print(f"Training for {EPOCHS} epochs")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Batch size: {BATCH_SIZE}")

In [ ]:
# Training loop
history = {'loss': [], 'emotion_acc': [], 'genre_acc': []}

print("\n" + "="*60)
print("TRAINING STARTED")
print("="*60)

for epoch in range(EPOCHS):
    model.train()
    total_loss = 0
    correct_emotion = 0
    correct_genre = 0
    total_samples = 0
    
    for batch in train_loader:
        # Move to device
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        emotion_labels = batch['emotion_label'].to(device)
        genre_labels = batch['genre_label'].to(device)
        
        # Forward pass
        optimizer.zero_grad()
        emotion_logits, genre_logits = model(input_ids, attention_mask)
        
        # Calculate loss
        emotion_loss = criterion(emotion_logits, emotion_labels)
        genre_loss = criterion(genre_logits, genre_labels)
        loss = emotion_loss + genre_loss
        
        # Backward pass (THIS IS THE LEARNING!)
        loss.backward()
        optimizer.step()
        
        # Track metrics
        total_loss += loss.item()
        
        emotion_preds = torch.argmax(emotion_logits, dim=-1)
        genre_preds = torch.argmax(genre_logits, dim=-1)
        
        correct_emotion += (emotion_preds == emotion_labels).sum().item()
        correct_genre += (genre_preds == genre_labels).sum().item()
        total_samples += len(emotion_labels)
    
    # Epoch stats
    avg_loss = total_loss / len(train_loader)
    emotion_acc = correct_emotion / total_samples * 100
    genre_acc = correct_genre / total_samples * 100
    
    history['loss'].append(avg_loss)
    history['emotion_acc'].append(emotion_acc)
    history['genre_acc'].append(genre_acc)
    
    print(f"Epoch {epoch+1}/{EPOCHS}")
    print(f"  Loss: {avg_loss:.4f}")
    print(f"  Emotion Acc: {emotion_acc:.1f}%")
    print(f"  Genre Acc: {genre_acc:.1f}%")
    print("-" * 40)

print("\n" + "="*60)
print("TRAINING COMPLETE! 🎉")
print("="*60)

## Step 9: Visualize Training Progress

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# Loss
axes[0].plot(history['loss'], 'b-o')
axes[0].set_title('Training Loss')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')

# Emotion Accuracy
axes[1].plot(history['emotion_acc'], 'g-o')
axes[1].set_title('Emotion Accuracy')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy (%)')
axes[1].set_ylim([0, 100])

# Genre Accuracy
axes[2].plot(history['genre_acc'], 'r-o')
axes[2].set_title('Genre Accuracy')
axes[2].set_xlabel('Epoch')
axes[2].set_ylabel('Accuracy (%)')
axes[2].set_ylim([0, 100])

plt.tight_layout()
plt.show()

## Step 10: Test the Trained Model

In [ ]:
def predict(prompt: str) -> dict:
    """Make prediction on a single prompt."""
    model.eval()
    
    # Tokenize
    encoding = tokenizer(
        prompt,
        truncation=True,
        padding='max_length',
        max_length=128,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    with torch.no_grad():
        emotion_logits, genre_logits = model(input_ids, attention_mask)
        
        emotion_probs = torch.softmax(emotion_logits, dim=-1)
        genre_probs = torch.softmax(genre_logits, dim=-1)
        
        emotion_idx = torch.argmax(emotion_probs, dim=-1).item()
        genre_idx = torch.argmax(genre_probs, dim=-1).item()
    
    return {
        'prompt': prompt,
        'emotion': IDX_TO_EMOTION[emotion_idx],
        'emotion_confidence': emotion_probs[0, emotion_idx].item(),
        'genre': IDX_TO_GENRE[genre_idx],
        'genre_confidence': genre_probs[0, genre_idx].item()
    }

In [ ]:
# Test prompts - including ambiguous ones!
test_prompts = [
    "sad folk song in E minor",
    "upbeat pop in G major",
    "something chill for the evening",  # Ambiguous - no explicit labels!
    "moody atmospheric vibes",           # Very ambiguous!
    "driving rock anthem",
    "peaceful acoustic morning coffee",
]

print("\n" + "="*60)
print("TESTING TRAINED MODEL")
print("="*60)

for prompt in test_prompts:
    result = predict(prompt)
    print(f"\nPrompt: '{result['prompt']}'")
    print(f"  → Emotion: {result['emotion']} ({result['emotion_confidence']:.1%})")
    print(f"  → Genre: {result['genre']} ({result['genre_confidence']:.1%})")

## Step 11: Save the Trained Model

In [ ]:
# Save model checkpoint
SAVE_PATH = 'neural_parser_checkpoint.pt'

torch.save({
    'model_state_dict': model.state_dict(),
    'emotion_to_idx': EMOTION_TO_IDX,
    'idx_to_emotion': IDX_TO_EMOTION,
    'genre_to_idx': GENRE_TO_IDX,
    'idx_to_genre': IDX_TO_GENRE,
    'history': history
}, SAVE_PATH)

print(f"Model saved to {SAVE_PATH}")

# Download the model
files.download(SAVE_PATH)

## Summary

You've just trained a **real neural network** for your thesis!

### What happened:
1. **DistilBERT** (66M parameters) converted your prompts to embeddings
2. **Classification heads** learned to predict emotion/genre from embeddings
3. **Backpropagation** adjusted weights based on your dataset labels
4. Model now understands **semantic meaning** of music descriptions

### Key differences from rule-based:
| Rule-Based | Neural (DistilBERT) |
|------------|---------------------|
| If-else logic | 66M learned parameters |
| Exact keyword match | Semantic understanding |
| "sad" → melancholic | "gloomy rainy day" → melancholic |
| Can't generalize | Handles unseen phrasings |

### For your thesis:
- This is the **AI/ML component** of the prompt parser
- Report training curves in your Results chapter
- Compare neural vs rule-based accuracy
- The hybrid system uses both!